In [1]:
%load_ext autoreload

The input dataset already contains a lot of engineered features. Moreover, I lack the expertise to suggest which features would make sense to engineer. For this reason, I decided not to engineer new features, but to focus on processing the exiting ones. The preprocessing pipeline that has been proposed in this project is the following:

In [49]:
%autoreload
from sklearn import set_config

from intrusion_detection.preprocessing.pipeline import get_preprocessing_pipeline
from intrusion_detection.preprocessing.preprocessing import remove_dot_from_attack_type_classes
from intrusion_detection.load_input_data import load_df
set_config(display="diagram")

In [50]:
preprocessing_pipeline = get_preprocessing_pipeline()
preprocessing_pipeline

Pipeline(steps=[('drop_target', DropFeatures(features_to_drop=['attack_type'])),
                ('outlier_removal',
                 Winsorizer(add_indicators=True, capping_method='quantiles',
                            fold=0.01, variables=['src_bytes', 'dst_bytes'])),
                ('frequency_encoder',
                 KeepInputFeaturesWrapper(rename_suffix='_freq',
                                          wrapped_transformer=CountFrequencyEncoder(encoding_method='frequency'))),
                ('replace_rare_categories',
                 RareLabelEncoder(n_categories=2, tol=0.01)),
                ('one_hot_encoder', OneHotEncoder()),
                ('min_max_scaler',
                 SklearnTransformerWrapper(transformer=MinMaxScaler()))])

## Numerical Features preprocessing

The first step of the pipeline is used to deal with outlier values. Similarly to what has been already mentioned in the exploratory data analysis section, this step will cap the features `src_bytes` and `dst_bytes` to the 99th percentile and will ad an indicator column specifying when the column values have been capped. All features (numerical and not) are scaled between 0 and 1 in the last step of the pipeline.

## Categorical Features Encoding

Categorical features are encoded in two ways, using frequency encoding and using OneHotEncoding. The latter creates a binary column for each category of each categorical variable, having `1` when the categorical variable has the category value. In order to limit the number of columns in our dataset, some rare categories (with frequency less than 1%) have been grouped into a single category called `Rare`. This has been obtained using the `RareLabelEncoder`. To counter the loss of information when replacing rare labels, categorical features have been also encoded using frequency encoding which replaces the category values with the relative frequencies of each category.



In [51]:
df = load_df(
    file_path="../../../data/kddcup.data_10_percent",
    header_file="../../../data/kddcup.names"
)
df = remove_dot_from_attack_type_classes(df)

In [53]:
preprocessed_df = preprocessing_pipeline.fit_transform(df)
preprocessed_df.describe().T

,count,mean,std,min,25%,50%,75%,max
duration,494021.0,0.000823,0.012134,0.0,0.000000,0.000000,0.000000,1.0
protocol_type_freq,494021.0,0.822097,0.241090,0.0,0.644681,1.000000,1.000000,1.0
service_freq,494021.0,0.689304,0.369005,0.0,0.394074,1.000000,1.000000,1.0
flag_freq,494021.0,0.810397,0.344788,0.0,1.000000,1.000000,1.000000,1.0
src_bytes,494021.0,0.257371,0.205174,0.0,0.018791,0.217137,0.430934,1.0
dst_bytes,494021.0,0.035271,0.141739,0.0,0.000000,0.000000,0.000000,1.0
land,494021.0,0.000045,0.006673,0.0,0.000000,0.000000,0.000000,1.0
wrong_fragment,494021.0,0.002144,0.044935,0.0,0.000000,0.000000,0.000000,1.0
urgent,494021.0,0.000005,0.001837,0.0,0.000000,0.000000,0.000000,1.0
hot,494021.0,0.001151,0.026070,0.0,0.000000,0.000000,0.000000,1.0
